<h2>Before you start</h2>
If this is a first time the pipeline is running on this machine, remove triple quotes and just run the cell below. This list of external libraries may not be full, in this case you may add required modules.

In [1]:
'''
!pip install moviepy
!pip install PySide6
!pip install wgpu glfw
!pip install fastplotlib
!pip install jupyter_rfb
!pip install sidecar
!pip install sortedcontainers
!pip install cmasher
!pip install cv2
'''

'\n!pip install moviepy\n!pip install PySide6\n!pip install wgpu glfw\n!pip install fastplotlib\n!pip install jupyter_rfb\n!pip install sidecar\n!pip install sortedcontainers\n!pip install cmasher\n!pip install cv2\n'

In [1]:
get_ipython().profile_dir.startup_dir

'C:\\Users\\admin\\.ipython\\profile_default\\startup'

<h2>Module 0</h2>
This module is obligatory, run the cell below each time you (re)start kernel. </br>IYou need to specify the root folder and folder_structure pattern, where to search for miniscope videos, typically '\\Mouse_name\\date\\time\Miniscope\\'. Note that * is a wildcard for any symbol combination except slashes (i.e., for any folder name), so it is strongly recommended to use it here.<br/><br/>
NB!! Just in case, use double backslashes for folder separation, otherwise some symbols may be interpreted as escape sequences. 

In [2]:
LOCAL_URLS

['http://localhost:8888/', 'http://localhost:8889/']

In [3]:
DATA_PATHWAY

'bonsai'

<h2>Module 1</h2>
Manual video inspection. <br/>Open folder with miniscopic videos in a pop-up window, wait for loading and specify margins to be cropped by sliders or by keyboard, then save them by running the next cell. At the time, cropping .pickle files are to be created in these folders. Repeat for all folders with miniscopic videos you would like to analyze.   

In [ ]:
#Manual file selection:
fnames = list(askopenfilenames(title = 'Select files for inspection', initialdir = root, filetypes = [('AVI files', '.avi')]))
#OR, alternatively, you can use Automatic file selection
#fnames = glob(root + folder_structure + '*.avi')

data = LoadSelectedVideos(fnames)
w = DrawCropper(data)

In [7]:
SaveCrops(os.path.dirname(fnames[0]), w.kwargs['left'], w.kwargs['right'], w.kwargs['up'], w.kwargs['down'])

{'LEFT': 50, 'RIGHT': 50, 'UP': 50, 'DOWN': 50}
Crops saved to C:\Users\Public\IABS_DATA\HM_NOF_2D\NOF_H02_2D\cropping.pickle



Batch cropping and timestamp extraction.<br/>Miniscopic videos from folders with .pickle files are to be cropped and saved as _CR.tif in the root folder. There is no need for renaming of sigle-digit .avi files (like 0-9.avi to 00-09.avi)!<br/>
Also, along with video data, timestamps are to be copied from minicopic folders to the root folder. Do not delete them, they are nessesary for the further steps!

In [36]:
#Batch crop
cpath_template = os.path.normpath(os.path.join(root, folder_structure, '*cropping.pickle'))
pick_names = glob(cpath_template)

for name in pick_names:
    DoCropAndRewrite(root, name, pathway=DATA_PATHWAY)
    extract_and_copy_ts(root, name, pathway=DATA_PATHWAY)

100%|██████████| 12/12 [00:24<00:00,  2.07s/it]


NOF_H01_2D_CR.tif cropped in 33.7s


100%|██████████| 12/12 [00:24<00:00,  2.01s/it]


NOF_H02_2D_CR.tif cropped in 32.9s


<h2>Module 2</h2>
Batch motion correction.<br/>All _CR.tif files in the root folder are to be automatically motion corrected with NoRMCorre routine [Pnevmatikakis, Giovanucci, 2017] with the parameters below and saved as _MC.tif files.

In [16]:
#Automatic file selection
fnames = glob(os.path.join(root, '*_CR.tif'))
#OR, alternatively, you can use manual file selection:
#fnames = askopenfilenames(title = 'Select files for motion correction', initialdir = root, filetypes = [('TIFF files', '.tif')])

mc_dict = {
    'pw_rigid': False,         # flag for performing piecewise-rigid motion correction (otherwise just rigid)
    'max_shifts': (35, 35),    # maximum allowed rigid shift
    'gSig_filt': (8, 8),       # size of high pass spatial filtering, used in 1p data
    'strides': (48, 48),       # start a new patch for pw-rigid motion correction every x pixels
    'overlaps': (24, 24),      # overlap between pathes (size of patch strides+overlaps)
    'max_deviation_rigid': 15,  # maximum deviation allowed for patch with respect to rigid shifts
    'border_nan': 'copy',      # replicate values along the boundaries
    'use_cuda': True,          # Set to True in order to use GPU
    'memory_fact': 4,          # How much memory to allocate. 1 works for 16Gb, so 0.8 showd be optimized for 12Gb.
    'niter_rig': 1,
    'splits_rig': 20,          # for parallelization split the movies in  num_splits chuncks across time
                               # if none all the splits are processed and the movie is saved
    'num_splits_to_process_rig': None} # intervals at which patches are laid out for motion correction  

for name in tqdm.tqdm(fnames):
    DoMotionCorrection(name, mc_dict)
    CleanMemmaps(name)

  0%|          | 0/2 [00:00<?, ?it/s]WARNING:root:Movie average is negative. Removing 1st percentile.

100%|██████████| 1/1 [00:04<00:00,  4.73s/it]


NOF_H01_2D_CR.tif motion corrected in 300.1s


 50%|█████     | 1/2 [05:02<05:02, 302.76s/it]WARNING:root:Movie average is negative. Removing 1st percentile.

100%|██████████| 1/1 [00:04<00:00,  4.38s/it]


NOF_H02_2D_CR.tif motion corrected in 308.8s


100%|██████████| 2/2 [10:13<00:00, 307.00s/it]


<h3>Module 2.5 (optional)</h3>
Pre-test of various values of <i>gSig</i> parameter, which is used in the Module 3 and corresponds to a typical radius of a neuron in pixels.<br/>You can play with this parameter but you can use the default value of gSig = 6 as well. <br/> Calculation may take a while, so be patient!

In [ ]:
fnames = glob(os.path.join(root, '*_MC.tif'))
#OR, alternatively, you can use manual file selection:
#fnames = askopenfilenames(title = 'Select files for gSig testing', initialdir = root, filetypes = [('TIFF files', '.tif')])

#Test_gSig_Range(fnames[0])
Test_gSig_Range(fnames[0], maxframes = 1000)  ## maxframes is the amount of frames taken into account, by default the whole file is to be taken, which may be too slow for large files

<h2>Module 3</h2>
Batch cnmf.<br/>All _MC.tif files in the root folder are to be automatically processed with CaImAn routine [Giovanucci et al., 2019] with the parameters below. Main parameters are gSig and gSiz for cell augmentation, then min_SNR as traces quality threshold. At the end, _estimates.pickle files are to be produced in the root folder. 

In [2]:
fnames = glob(os.path.join(root, '*_MC.tif'))
#OR, alternatively, you can use manual file selection:
#fnames = askopenfilenames(title = 'Select files for batch cnmf', initialdir = root, filetypes = [('TIFF files', '.tif')])

cnmf_dict= {'fr': 20,                   # frame rate, frames per second (NOW RECALCULATED FOR EACH FILE FROM TIMESTAMP DATA)
            'decay_time': 2,            # typical duration of calcium transient 
            'method_init': 'corr_pnr',  # use this for 1 photon
            'K': None,                  # upper bound on number of components per patch, in general None
            'gSig': (6, 6),             # gaussian HALF-width of a 2D gaussian kernel (in pixels), which approximates a neuron
            'gSiz': (10, 10),           # maximal radius of a neuron in pixels
            'merge_thr': 0.85,          # merging threshold, max correlation allowed
            'p': 1,                     # order of the autoregressive system
            'tsub': 1,                  # downsampling factor in time for initialization
            'ssub': 1,                  # downsampling factor in space for initialization
            'rf': 30,                   # half-size of the patches in pixels. e.g., if rf=40, patches are 80x80
            'stride': 25,               # amount of overlap between the patches in pixels(keep it at least large as gSiz, i.e 4 times the neuron size gSig) 
            'only_init': True,          # set it to True to run CNMF-E
            'nb': 0,                    # number of background components (rank) if positive, else exact ring model with following settings: nb= 0: Return background as b and W, gnb=-1: Return full rank background B, gnb<-1: Don't return background
            'nb_patch': 0,              # number of background components (rank) per patch if nb>0, else it is set automatically
            'method_deconvolution': 'oasis',       # could use 'cvxpy' alternatively
            'low_rank_background': None,           # None leaves background of each patch intact, True performs global low-rank approximation if gnb>0
            'update_background_components': True,  # sometimes setting to False improve the results
            'min_corr': .8,                        # min peak value from correlation image
            'min_pnr': 10,                         # min peak to noise ratio from PNR image
            'normalize_init': False,               # just leave as is
            'center_psf': True,                    # leave as is for 1 photon
            'ssub_B': 2,                           # additional downsampling factor in space for background
            'ring_size_factor': 1.5,               # radius of ring is gSiz*ring_size_factor
            'del_duplicates': True,                # whether to remove duplicates from initialization
            'border_pix': 5,                       # number of pixels to not consider in the borders
            'min_SNR': 1,                          # adaptive way to set threshold on the transient size
            'rval_thr': 0.5,                       # threshold on space consistency           
            'use_cnn': False}                      # whether to use CNNs for event detection  


start_frame = 100
end_frame = 300

for gsig in [6,7,8]:
    cnmf_dict.update({'gSig': (gsig, gsig)})
    
    for name in tqdm.tqdm(fnames):
        fps = get_fps_from_timestamps(name[:-4-6], default_fps=20, verbose=False)
        cnmf_dict.update({'fr': fps})
        out_name = name[:-4] + f'_gsig={gsig}'
        
        DoCNMF(name,
               cnmf_dict,
               out_name=out_name,
               start_frame=start_frame,
               end_frame=end_frame,
               verbose=True)
        
        CleanMemmaps(name)

  0%|          | 0/2 [00:00<?, ?it/s]

loading tif to memory...
Performing CNMF...
Computing imax...
Saving result...


 50%|█████     | 1/2 [01:17<01:17, 77.32s/it]

NOF_H01_2D_CR_MC.tif cnmf-ed in 77.2s
loading tif to memory...
Performing CNMF...
Computing imax...
Saving result...


100%|██████████| 2/2 [03:01<00:00, 90.90s/it]


...

loading tif to memory...
Performing CNMF...
Computing imax...
Saving result...


 50%|█████     | 1/2 [01:24<01:24, 84.97s/it]

NOF_H01_2D_CR_MC.tif cnmf-ed in 84.9s
loading tif to memory...
Performing CNMF...
Computing imax...
Saving result...


100%|██████████| 2/2 [03:19<00:00, 99.62s/it] 

NOF_H02_2D_CR_MC.tif cnmf-ed in 114.2s


<h2>Module 4</h2>
User inspection of cnmf results.<br/>
Btw, at this stage, previously saved timestamps are to be merged with cnmf results.
By running the section below, you will be prompted to select estimtes file with cnmf results and then to interactively examine detected units (you can select both spatial and temporal components), you can select, merge and delete them, also you can seed new neurons (by PointDrawTool) for further re-run of CNMF with saved seeds. Finally, you can save (by pressing 'Save Results') spatial and temporal components as .tif and traces.csv files, respectively. Spatial components (aka filters) are to be stored in a separate folder (*_filters).

In [ ]:
fname = askopenfilename(title = 'Select estimates file for examination', initialdir = root, filetypes = [('estimates files', '*estimates.pickle')])

bkapp_kwargs = {
    'pathway': DATA_PATHWAY, 
    'fill_alpha': 0.5,       # selected neuron transparency
    'ns_alpha': 0.2,         # non-selected neuron transparency
    'line_width': 2,         # border width
    'cthr': 0.8              # coutour_thr from caiman (% of signal inside a patch), affects patch size
}

ExamineCells(fname, default_fps=20, bkapp_kwargs=bkapp_kwargs)

In [ ]:
#Use this cell in case of "Refusing websocket connection..." error in the cell above
os.environ["BOKEH_ALLOW_WS_ORIGIN"] = 'localhost:8888' #replace the port number with the actual one from the address string of your browser

Redo cnmf with manually added seeds (optional).<br/>
NB!! By running the cell below, you will rewrite existing estimates files!!<br/>
Then you can return to the section above and inspect the rewritten estimates.

In [ ]:
s_names = glob(os.path.join(root, '*seeds.pickle'))
#OR, alternatively, you can use manual file selection:
#s_names = askopenfilenames(title = 'Select seeds files for re-CNMFing', initialdir = root, filetypes = [('seeds files', '*seeds.pickle')])

for s_name in s_names:
    e_names = glob(s_name.partition('_seeds')[0] + '_estimates.pickle')
    ReDoCNMF(s_name, e_names[0])
    CleanMemmaps(s_name.partition('_seeds')[0])

<h2>Module 5</h2>
Batch event detection. <br/>
INPUT: (timestamped) cnmf raw traces as *_traces.csv files<br/>
OUTPUT: detected events as *_spikes.csv files; pickles with events (cell-wise list of event-wise lists with dictionaries) and also, interactive .html plot with traces and events.

In [ ]:
fnames = glob(root + '*traces.csv')
#OR, alternatively, you can use manual file selection:
#fnames = askopenfilenames(title = 'Select traces for event detection', initialdir = root, filetypes = [('traces files', '*traces.csv')])

sd_dict = {'thr': 4,        #threshold for peaks in Median Absolute Deviations (MADs)                   
           'sigma' : 7,     #smoothing parameter for peak detection, frames
           'est_ton' : 0.5, #estimated event rising time, s
           'est_toff' : 2,  #estimated event decay time, s
           'draw_details': True} #whether to draw smoothed traces, peaks, pits and fits 

for name in fnames:
    FitEvents(name, opts = sd_dict)


Also, just in case, you may draw existed pairs of traces and spikes right here:

In [ ]:
fnames = glob(root + '*traces.csv')
for name in fnames:
    DrawSpEvents(name, name.replace('traces','spikes'))
